# 세이지메이커 모델 병렬성 라이브러리에서 Sharded Data Parallelism 기법을 활용해 Falcon 모델 학습

이 노트북에서는 허깅 페이스 트랜스포머 [Falcon](https://huggingface.co/docs/transformers/main/model_doc/falcon) 모델을 파이토치 2.0과 [세이지메이커의 모델 병렬성 라이브러리 (SMP)](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html)에서 지원하는 [샤딩 데이터 병렬성(Sharded Data Parallelism)](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-sharded-data-parallelism.html) 기법을 활용해 [GLUE/SST2 데이터 세트](https://huggingface.co/datasets/glue/viewer/sst2/train)로 학습하는 방법을 배울 수 있습니다.

샤딩 데이터 병렬성은 모델 파라미터, 기울기, 옵티마이저 상태를 데이터 병렬 그룹의 GPU 간에 분산시키는 분산 학습 기법입니다. 이 기법은 극단적으로 큰 모델을 위해 설계되었으며, 아마존의 자체 기술인 [MiCS](https://arxiv.org/pdf/2205.00119.pdf)를 활용해 거의 선형적인 확장 효율성을 달성합니다. 단일 GPU에 맞지 않는 대형 모델의 경우, 먼저 SMP안에서 샤딩 데이터 병렬성 기법과 함께 [Activation Checkpointing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-activation-checkpointing.html) 및 [Activation Offloading](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-activation-offloading.html)을 활용하는 것을 추천합니다. 이후에는 텐서 병렬성 또는 파이프라인 병렬성과 같은 다른 기법을 고려할 수 있습니다.

이 기능은 [텐서 병렬성](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-tensor-parallelism.html)과도 호환됩니다.

이 노트북에는 다음 파일이 포함되어 있습니다.

- `train.py`: 세이지메이커 파이토치 에스티메이터에 전달될 엔트리 포인트 스크립트입니다. 이 스크립트는 SMP와 함께 Falcon 모델의 종단 간 학습을 실행하며, 샤딩 데이터 병렬성 설정이 적용된 코드로 모델을 저장, 로드 및 미세 조정하는 기능이 구현되어 있습니다.
- `data_pipeline.py`: 학습 데이터 세트를 준비하기 위한 데이터 파이프라인 함수가 포함되어 있습니다.
- `learining_rate.py`: 학습률 스케줄을 위한 함수가 포함되어 있습니다.
- `requirements.txt`: 허깅 페이스 트랜스포머를 포함한 의존성을 설치합니다.
- `memory_tracker.py`: 메모리 사용량을 추적하는 함수가 포함되어 있습니다.
- `model_config.py`: 모델 구성 정보를 가져오는 함수가 포함되어 있습니다.
- `sdp_utils.py`: 샤딩 데이터 병렬성을 위한 유틸리티 함수가 포함되어 있습니다.

### 추가 자료
- 세이지메이커 모델 병렬성 라이브러리에 대한 더 자세한 내용은 [세이지메이커 분산을 활용한 모델 병렬 분산 학습](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html) 문서에서 확인할 수 있습니다.

- 세이지메이커 파이썬 SDK를 파이토치와 함께 활용하는 방법에 대한 더 자세한 내용은 [세이지메이커 파이썬 SDK와 파이토치 활용](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html) 문서에서 확인할 수 있습니다.

- 자게 학습 이미지를 활용해 아마존 세이지메이커에서 학습 작업을 시작하는 방법에 대한 더 자세한 내용은 [자체 학습 알고리즘 활용](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) 문서에서 확인할 수 있습니다.

- 샤딩 데이터 병렬성에 대한 더 자세한 내용은 [샤딩 데이터 병렬성](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-sharded-data-parallelism.html) 문서나 블로그 [AWS에서 거대 모델 학습의 선형적 확장](https://www.amazon.science/blog/near-linear-scaling-of-gigantic-model-training-on-aws)에서 확인할 수 있습니다.

### 사전 요구 사항
학습을 위한 입력 데이터를 저장할 S3 버킷을 생성해야 합니다. 이 버킷은 학습 작업을 시작할 AWS 리전과 동일한 리전에 있어야 합니다. S3 버킷을 생성하는 방법에 대한 더 자세한 내용은 *아마존 S3 문서*의 [첫 번째 S3 버킷 생성](https://docs.aws.amazon.com/AmazonS3/latest/userguide/creating-bucket.html)에서 확인할 수 있습니다.

## 아마존 세이지메이커 초기화

다음 셀을 실행하여 세이지메이커 모듈을 가져오고 현재 세이지메이커 작업 환경에 대한 정보를 조회합니다. 여기에는 AWS 계정 ID, AWS 리전, 아마존 세이지메이커 실행 역할의 ARN이 포함됩니다. 세이지메이커 SDK를 최신 버전으로 업그레이드합니다.

**참고:** 이 단계에서는 커널 재시작이 필요할 수 있습니다.

In [ ]:
%pip install --upgrade sagemaker
%pip install sagemaker-experiments

In [ ]:
%%time
import os

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

role = (
    get_execution_role()
)  # 새 역할을 생성하는 대신 기존 역할 ARN 제공
print(f"SageMaker Execution Role: {role}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: {account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: {region}")

sm_boto_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session(boto_session=session)

# 기본 버킷 가져오기
default_bucket = sagemaker_session.default_bucket()
print()
print("Default bucket for this session: ", default_bucket)

## GLUE/SST2 데이터 다운로드 및 준비
여기에서는 GLUE/SST2 데이터 세트를 다운로드하고 준비한 후, 파일을 S3에 복사합니다.

### 허깅 페이스 트랜스포머와 데이터 세트 라이브러리 설치

In [ ]:
! pip install -q datasets transformers==4.21.0

In [ ]:
import datasets
from datasets import load_dataset, load_from_disk, load_metric

In [ ]:
from sagemaker.pytorch import PyTorch
import transformers
import logging

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

from transformers.testing_utils import CaptureLogger

In [ ]:
logger = logging.getLogger(__name__)

### 데이터 로드
이 섹션에서는 [GLUE/SST2](https://huggingface.co/datasets/glue/viewer/sst2/train) 데이터 세트를 로드하고 학습 및 검증 데이터 세트로 나눕니다.

In [ ]:
hyperparameters = {
    "dataset_name": "glue",
    "dataset_config_name": "sst2",
    "do_train": True,
    "do_eval": True,
    "cache_dir": "tmp",
}

In [ ]:
raw_datasets = load_dataset(
    hyperparameters["dataset_name"],
    hyperparameters["dataset_config_name"],
)

In [ ]:
if "validation" not in raw_datasets.keys():
    raw_datasets["validation"] = load_dataset(
        hyperparameters["dataset_name"],
        hyperparameters["dataset_config_name"],
        split="train[:5%]",
        cache_dir=hyperparameters["cache_dir"],
    )

    raw_datasets["train"] = load_dataset(
        hyperparameters["dataset_name"],
        hyperparameters["dataset_config_name"],
        split="train[5%:]",
        cache_dir=hyperparameters["cache_dir"],
    )

### 토크나이저 로드

거의 모든 NLP 작업은 토크나이저로 시작합니다. 토크나이저는 텍스트 데이터를 NLP 모델이 처리할 수 있는 형식(토큰)으로 변환합니다.
다음 셀에서는 [AutoTokenizer.from_pretrained()](https://huggingface.co/docs/transformers/v4.19.4/en/autoclass_tutorial#autotokenizer)를 활용해 Falcon 모델의 토크나이저를 로드합니다.

In [ ]:
tokenizer_kwargs = {
    "cache_dir": hyperparameters["cache_dir"],
}

tokenizer = AutoTokenizer.from_pretrained(
    "tiiuae/falcon-40b", trust_remote_code=True, **tokenizer_kwargs
)

### 데이터 전처리

다음 두 셀에서는 토크나이저를 실행하고 텍스트를 블록 크기보다 작은 청크로 그룹화하는 함수를 설정합니다.

In [1]:
def tokenize_function(examples):
    tok_logger = transformers.utils.logging.get_logger("transformers.tokenization_utils_base")

    with CaptureLogger(tok_logger) as cl:
        output = tokenizer(examples[text_column_name])
        # clm 입력은 블록 크기보다 훨씬 더 길 수 있습니다.
        if "Token indices sequence length is longer than the" in cl.out:
            tok_logger.warning(
                "^^^^^^^^^^^^^^^^ Please ignore the warning above - this long input will be chunked into smaller bits before being passed to the model."
            )
    return output


# 데이터 세트의 모든 텍스트를 연결하고 블록 크기의 청크를 생성하는 주요 데이터 처리 함수입니다.
def group_texts(examples):
    # 모든 텍스트를 연결합니다.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # 작은 나머지는 삭제합니다. 모델이 패딩을 지원하면 이 삭제 대신 패딩을 추가할 수 있습니다.
    # 필요에 따라 이 부분을 사용자 맞춤 설정할 수 있습니다.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
        # max_len으로 청크로 분할합니다.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

# _LazyModule 오류를 피하기 위해 이 함수를 피클링할 때 토크나이즈 함수 전에 로거 로딩 강제
tok_logger = transformers.utils.logging.get_logger("transformers.tokenization_utils_base")

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=1,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)


block_size = tokenizer.model_max_length
if block_size > 1024:
    logger.warning(
        f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
        "Picking 1024 instead. You can change that default value by passing --block_size xxx."
    )
    block_size = 1024
else:
    if block_size > tokenizer.model_max_length:
        logger.warning(
            f"The block_size passed ({block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(block_size, tokenizer.model_max_length)

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    #     num_proc=args.preprocessing_num_workers,
    desc=f"Grouping texts in chunks of {block_size}",
)

학습 작업의 각 에포크에서 학습 및 검증 데이터 세트를 적절히 매핑할 수 있도록 추가 하이퍼파라미터 및 S3 경로를 설정합니다.

In [ ]:
if hyperparameters["do_train"]:
    if "train" not in tokenized_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = lm_datasets["train"]


if hyperparameters["do_eval"]:
    if "validation" not in tokenized_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_dataset = lm_datasets["validation"]

In [ ]:
training_dataset_location = None
validation_dataset_location = None


if hyperparameters["do_train"]:
    train_dataset.to_json("./training.json")
    training_dataset_location = "s3://{}/dataset/train/".format(default_bucket)

if hyperparameters["do_eval"]:
    eval_dataset.to_json("./validation.json")
    validation_dataset_location = "s3://{}/dataset/validation/".format(default_bucket)

In [ ]:
if training_dataset_location is not None:
    command = "aws s3 cp ./training.json {}".format(training_dataset_location)
    os.system(command)

if validation_dataset_location is not None:
    command = "aws s3 cp ./validation.json {}".format(validation_dataset_location)
    os.system(command)

In [ ]:
if hyperparameters["do_train"]:
    command = "rm ./training.json"
    os.system(command)

if hyperparameters["do_eval"]:
    command = "rm ./validation.json"
    os.system(command)

In [ ]:
%store training_dataset_location
%store validation_dataset_location

In [ ]:
%store

## 아마존 S3 버킷 경로 지정

여기에서는 작업에 활용할 학습 데이터 경로를 지정해야 합니다. 사용되는 버킷은 학습이 실행될 리전과 동일해야 합니다. 위의 셀에서 GLUE/SST2 학습 및 검증 분할 데이터 세트를 다운로드하고 JSON 파일을 S3 버킷에 업로드했습니다. 이 예제는 이 JSON 파일에서 학습을 진행합니다.

예제 텐서 병렬 학습 작업이 성공적으로 실행된 후, 자신만의 데이터 세트가 저장된 S3 버킷으로 수정할 수 있습니다.

In [ ]:
%store -r training_dataset_location
%store -r validation_dataset_location

# 자체 데이터를 사용하는 경우, 아래 줄의 주석을 제거하고 위치를 지정할 수 있습니다.
# training_dataset_location = YOUR_S3_BUCKET/training
# validation_dataset_location = YOUR_S3_BUCKET/validation

In [ ]:
s3_train_bucket = training_dataset_location
s3_test_bucket = validation_dataset_location

다음 S3 버킷은 학습 작업의 출력 아티팩트를 저장합니다. 필요에 따라 수정할 수 있습니다.

In [ ]:
s3_output_bucket = f"s3://sagemaker-{region}-{account}/smp-tensorparallel-outputdir/"

## 아마존 S3로 세이지메이커 학습 데이터 채널 정의

이 단계에서는 세이지메이커 학습 데이터 채널을 S3 버킷에 정의합니다.

In [ ]:
# use_fsx 기본값은 False로 설정
# fsx를 사용하려는 경우, 아래 변수를 True로 설정 (다음 셀 참조)
use_fsx = False
if not use_fsx:
    if s3_train_bucket != None:
        train = sagemaker.inputs.TrainingInput(
            s3_train_bucket, distribution="FullyReplicated", s3_data_type="S3Prefix"
        )
        data_channels = {"train": train}
    else:
        data_channels = {"train": mock_data}
    if s3_test_bucket != None:
        test = sagemaker.inputs.TrainingInput(
            s3_test_bucket, distribution="FullyReplicated", s3_data_type="S3Prefix"
        )
        data_channels["test"] = test
    else:
        data_channels["test"] = mock_data
    print(data_channels)

## (선택 사항) 아마존 FSx를 데이터 채널 및 체크포인트 사용 설정

이전 옵션인 아마존 S3 사용은 설정이 더 쉬우나, 대용량 입력 및 모델 사이즈를 처리할 때 FSx를 사용하는 것이 성능에 유리할 수 있습니다. 13B 이상의 모델을 사용하는 경우 체크포인팅은 FSx를 사용해야 합니다.

FSx Lustre 파일 시스템을 생성하고 S3 버킷에서 FSx 파일 시스템으로 데이터 세트를 가져오는 방법은 [아마존 세이지메이커에서 FSx를 사용한 Mask-RCNN의 분산 학습](https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb)에서 확인할 수 있습니다. FSx 파일 시스템은 학습 작업이 인터넷에 접근할 수 있도록 인터넷 게이트웨이가 있는 프라이빗 서브넷에서 생성돼야 합니다. FSx Lustre 파일 시스템을 데이터 입력 채널로 설정하는 방법에 대한 일반적인 안내는 [아마존 FSx Lustre를 사용하는 데이터 입력 채널 구성 문서](https://docs.aws.amazon.com/sagemaker/latest/dg/model-access-training-data.html#model-access-training-data-fsx)에서 확인할 수 있습니다.

In [ ]:
# 다음에서 얻은 지침:
# https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb

if use_fsx:
    from sagemaker.inputs import FileSystemInput

    # FSx Lustre 파일 시스템 ID를 지정합니다.
    file_system_id = "<your-file-system-id>"

    # FSx에 활용된 보안 그룹(SG)과 서브넷을 지정합니다. 이러한 정보는 세이지메이커 에스티메이터에 전달돼 작업에서 활용됩니다.
    fsx_security_group_id = "<your-security-group-id>"
    fsx_subnet = "<your-subnet>"

    # 파일 시스템의 입력 데이터 디렉터리 경로를 지정합니다.
    # 아래에 정규화된 절대 경로를 제공해야 합니다.
    # fsx 생성 시 또는 자동으로 생성된 마운트 이름을 사용할 수 있습니다.
    # 이 마운트 이름은 콘솔의 FSX 페이지에서 확인할 수 있습니다.
    # fsx에서 생성된 마운트 이름 예: "3x5lhbmv"
    base_path = "<your-mount-name>"

    # 파일 시스템 유형을 지정합니다.
    file_system_type = "FSxLustre"

    train = FileSystemInput(
        file_system_id=file_system_id,
        file_system_type=file_system_type,
        directory_path=base_path,
        file_system_access_mode="rw",
    )

    data_channels = {"train": train, "test": train}

## 하이퍼파라미터, 메트릭 정의 및 MPI 옵션 설정

다음 `hyperparameters` 딕셔너리는 학습 스크립트(`train.py`)에 인수를 전달하고 학습 작업을 생성할 때 모델 병렬 구성을 설정합니다.

커스텀 `mpi` 플래그를 추가할 수도 있습니다. 기본적으로 `--mca btl_vader_single_copy_mechanism none`을 활용해 불필요한 로그를 제거합니다.

다음으로, 세이지메이커에서 메트릭 업로드를 활성화하기 위해 기본 메트릭 정의를 추가합니다. 추가적인 메트릭 정의를 추가할 수 있습니다.

`sharded_data_parallel_degree` 매개변수를 `hyperparameter` 딕셔너리에 추가하는 점을 주의깊게 확인해야 합니다. 이 매개변수는 세이지메이커 파이토치 에스티메이터를 구성할 때 샤딩 데이터 병렬성을 활성화하는 데 활용됩니다.

In [ ]:
hyperparameters = {
    "max_steps": 100,
    "seed": 12345,
    "fp16": 0,
    "bf16": 1,
    "lr": 2.0e-4,
    "lr_decay_iters": 125000,
    "min_lr": 0.00001,
    "lr-decay-style": "linear",
    "warmup": 0.01,
    "num_kept_checkpoints": 5,
    "checkpoint_freq": 200,
    "logging_freq": 1,
    "save_final_full_model": 0,
    "delayed_param": 1,
    "offload_activations": 0,
    "activation_loading_horizon": 4,
    "gradient_accumulation": 1,
    "validation_freq": 200,
    "train_batch_size": 4,
    "val_batch_size": 4,
    "zipped_data": 0,
    "epochs": 100,
    "use_distributed_transformer": 0,
    "model_type": "falcon",
    # 샤딩 데이터 병렬성을 위한 매개변수
    "sharded_data_parallel_degree": 16,
}

if use_fsx:
    # fsx 데이터 세트 경로에 따라 training-dir 및 test-dir 경로를 업데이트해야 합니다. 
    # 학습을 다시 시작하려는 경우 checkpoint-dir를 이전 작업과 동일한 경로로 설정합니다.
    SM_TRAIN_DIR = "/opt/ml/input/data/train"
    hyperparameters["checkpoint-dir"] = f"{SM_TRAIN_DIR}/checkpointdir-job2"
    hyperparameters["model-dir"] = f"{SM_TRAIN_DIR}/modeldir-job2"
    hyperparameters["training-dir"] = f"{SM_TRAIN_DIR}/datasets/pytorch_gpt/train_synthetic"
    hyperparameters["test-dir"] = f"{SM_TRAIN_DIR}/datasets/pytorch_gpt/val_synthetic"

# 체크포인트 경로 (hyperparameters['checkpoint-dir'] 또는 checkpoint_s3_uri)는 작업마다 고유하지 않습니다.
# 서로 다른 실행을 위해 필요에 따라 수정해야 합니다.
# 동일한 경로를 관련 없는 실행에 사용할 경우, 불필요한 체크포인트를 다운로드하는 시간이 늘어나고,
# 체크포인트를 로드할 때 충돌이 발생할 수 있습니다.

mpioptions = "-x NCCL_DEBUG=WARN -x SMDEBUG_LOG_LEVEL=ERROR "
mpioptions += (
    "-x SMP_DISABLE_D2D=1 -x SMP_D2D_GPU_BUFFER_SIZE_BYTES=1 -x SMP_NCCL_THROTTLE_LIMIT=1 "
)
mpioptions += "-x FI_EFA_USE_DEVICE_RDMA=1 -x FI_PROVIDER=efa -x RDMAV_FORK_SAFE=1"

metric_definitions = [
    {"Name": "base_metric", "Regex": "<><><><><><>"}
]  # 사용자 정의 메트릭 정의를 추가합니다.

모델 구성을 설정합니다.

In [ ]:
model_config = "falcon-7b"

if model_config == "falcon-7b":
    model_params = {
        "max_context_width": 2048,
        "hidden_width": 4544,
        "num_layers": 32,
        "num_heads": 71,
        "num_heads_kv": 71,
    }
else:
    raise RuntimeError("Unknown model config")

for k, v in model_params.items():
    hyperparameters[k] = v

## 세이지메이커 학습 작업에 대한 필수 매개변수 지정

다음으로, [세이지메이커 에스티메이터 클래스](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)를 활용하여 세이지메이커 학습 작업을 정의하며, 학습 작업 이름, EC2 인스턴스 수, 인스턴스 유형 및 인스턴스에 연결된 볼륨 크기와 같은 값을 다음 매개변수를 통해 전달합니다.

* `instance_count`
* `instance_type`
* `volume_size`
* `base_job_name`

### EC2 인스턴스 유형 및 갯수 업데이트

`instance_type` 및 `instance_count` 매개변수에 지정하는 인스턴스 유형과 인스턴스 수는 총 GPU 수(world size)를 결정합니다.

$$ \text{(world size) = (단일 인스턴스의 GPU 수)}\times\text{(인스턴스 수)}$$

In [ ]:
instance_type = "ml.p4d.24xlarge"

instance_count = 2

# 해당 인스턴스의 GPU 수를 설정합니다.
processes_per_host = 8

다양한 인스턴스 유형의 GPU 수를 확인하려면 [아마존 EC2 인스턴스 유형](https://aws.amazon.com/ec2/instance-types/)을 확인합니다. **Accelerated Computing** 섹션에서 일반 목적 GPU 인스턴스를 확인할 수 있습니다. 예를 들어, `p4d.24xlarge`라는 인스턴스 유형은 세이지메이커에서 `ml.p4d.24xlarge`에 해당합니다.
세이지메이커에서 지원하는 `ml` 인스턴스와 비용 정보는 [아마존 세이지메이커 비용](https://aws.amazon.com/sagemaker/pricing/)에서 확인할 수 있습니다.

### 기본 작업 이름 지정

In [ ]:
machine_str = instance_type.split(".")[1] + instance_type.split(".")[2][:3]
sharding_degree = hyperparameters["sharded_data_parallel_degree"]
base_job_name = (
    f'smp-{model_config}-{machine_str}-sdp{sharding_degree}-bs{hyperparameters["train_batch_size"]}'
)

In [ ]:
if not use_fsx:
    # 학습을 다시 시작하려면 checkpoint_s3_uri를 이전 작업과 동일한 경로로 설정합니다.
    # 로드할 이전 체크포인트는 동일한 모델 구성이어야 합니다.
    checkpoint_bucket = f"s3://sagemaker-{region}-{account}/"
    checkpoint_s3_uri = (
        f"{checkpoint_bucket}/experiments/gpt_synthetic_simpletrainer_checkpoints/{base_job_name}/"
    )

In [ ]:
print(f"base_job_name: {base_job_name} checkpoint_s3_uri: {checkpoint_s3_uri}")

### 세이지메이커 파이토치 에스티메이터 생성

다음 셀에서는 위에서 정의한 매개변수로 파이토치 에스티메이터를 생성합니다. 세이지메이커 API와 함수가 스크립트에 어떻게 적용되는지 보려면 `train.py` 파일을 참조하면 좋습니다.


In [ ]:
kwargs = {}
if use_fsx:
    # FSx 파일 시스템을 생성할 때 활용한 보안 그룹과 서브넷 설정
    kwargs["security_group_ids"] = [fsx_security_group_id]
    kwargs["subnets"] = [fsx_subnet]

smp_estimator = PyTorch(
    entry_point="train.py",
    source_dir=os.getcwd(),
    role=role,
    instance_type=instance_type,
    instance_count=instance_count,
    sagemaker_session=sagemaker_session,
    distribution={
        "mpi": {
            "enabled": True,
            "processes_per_host": processes_per_host,
            "custom_mpi_options": mpioptions,
        },
        "smdistributed": {
            "modelparallel": {
                "enabled": True,
                "parameters": {
                    "ddp": True,
                    "skip_tracing": True,
                    "delayed_parameter_initialization": hyperparameters["delayed_param"] > 0,
                    "offload_activations": hyperparameters["offload_activations"] > 0,
                    "activation_loading_horizon": hyperparameters["activation_loading_horizon"],
                    "sharded_data_parallel_degree": hyperparameters["sharded_data_parallel_degree"],
                    "fp16": hyperparameters["fp16"] > 0,
                    "bf16": hyperparameters["bf16"] > 0,
                    # partitions는 현재 SM SDK에서 필수 매개변수이므로 반드시 전달해야 합니다.
                    "partitions": 1,
                },
            }
        },
    },
    framework_version="2.0",
    py_version="py310",
    output_path=s3_output_bucket,
    checkpoint_s3_uri=checkpoint_s3_uri if not use_fsx else None,
    checkpoint_local_path=hyperparameters["checkpoint-dir"] if use_fsx else None,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    debugger_hook_config=False,
    disable_profiler=True,
    base_job_name=base_job_name,
    **kwargs,
)

Finally, run the `estimator.fit` method to launch the SageMaker training job of the Falcon model with sharded data parallelism.

In [ ]:
smp_estimator.fit(inputs=data_channels, logs=True)

## 학습 로그 접근

학습 로그는 [아마존 클라우드워치](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/WhatIsCloudWatch.html)에서 접근할 수 있습니다. 학습 작업 로그가 포함된 출력 스트림을 가진 **algo-1** 노드의 로그를 확인해야 합니다.

클라우드워치를 활용해 학습 및 추론 동안 세이지메이커 GPU 및 메모리 사용량을 추적할 수 있습니다. 세이지메이커가 클라우드워치에 기록하는 메트릭 및 로그에 대한 자세한 내용은 아마존 세이지메이커 개발자 가이드의 [세이지메이커 작업 및 엔드포인트 매트릭](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html#cloudwatch-metrics-jobs) 섹션에서 확인할 수 있습니다.

클라우드워치를 처음 사용하는 경우, [아마존 클라우드워치 시작하기](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/GettingStarted.html) 문서를 확인하면 좋습니다.

아마존 세이지메이커 학습 작업을 모니터링하고 분석하는 추가 정보는 [메트릭을 활용해 학습 작업 모니터링 및 분석하기](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html) 문서에서 확인할 수 있습니다.


## 추론을 위한 학습 모델 배포

대부분의 경우, 학습된 모델은 추론을 위해 단일 장치에 배포할 수 있습니다. 추론은 적은 메모리만 필요하기 때문입니다. 학습 후에는 SMP API를 활용해 단일 통합 모델을 생성할 수 있습니다: 텐서플로우의 경우 [smp.DistributedModel.save_model()](https://sagemaker.readthedocs.io/en/stable/api/training/smp_versions/latest/smd_model_parallel_tensorflow.html#smp.DistributedModel.save_model) 함수, 파이토치의 경우 [smp.save()](https://sagemaker.readthedocs.io/en/stable/api/training/smp_versions/latest/smd_model_parallel_pytorch.html#apis-for-saving-and-loading) 함수를 사용하면 됩니다.

모델을 빌드하고 학습한 후, 다음 두 가지 방법 중 하나로 배포하여 예측 결과를 수신할 수 있습니다.

* 모델의 예측 결과를 수신하기 위한 엔드포인트를 설정하려면 세이지메이커 호스팅 서비스를 사용하면 됩니다. 세이지메이커 호스팅 서비스를 활용해 단일 모델 또는 여러 모델을 배포하는 개요는 [세이지메이커 호스팅 서비스에서 모델 배포하기](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-deployment.html#how-it-works-hosting)에서 확인할 수 있습니다.
* 전체 데이터 세트에 대한 예측 결과를 수신하려면 세이지메이커 배치 트랜스폼을 사용하면 됩니다. 세이지메이커 배치 트랜스폼을 활용해 모델을 배포하는 개요는 [배치 트랜스폼으로 전체 데이터 세트에 대한 추론 받기](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)에서 확인할 수 있습니다.

세이지메이커를 활용해 추론을 위해 모델을 배포하는 방법에 대해 더 알고 싶다면, [추론을 위한 모델 배포](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html) 문서에서 확인할 수 있습니다.